# Pobieranie danych

In [1]:
!gdown http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip
!unzip MachineLearningCSV.zip

Downloading...
From: http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip
To: /content/MachineLearningCSV.zip
100% 235M/235M [02:06<00:00, 1.86MB/s]
Archive:  MachineLearningCSV.zip
   creating: MachineLearningCVE/
  inflating: MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv  


# Importy i ważne zmienne


In [2]:
import numpy as np
import pandas as pd
import os


from fastai.tabular.all import df_shrink
dir_path = './MachineLearningCVE'


# Czytanie ścieżek do plików i danych

In [3]:
def read_paths(dir_path):
  dspaths = []
  for dirname, _, filenames in os.walk(dir_path):
      for filename in filenames:
          if filename.endswith('.csv'):
              pds = os.path.join(dirname, filename)
              dspaths.append(pds)
              print(pds)
  return dspaths

def read_files(dspaths):
  individual_dfs = [pd.read_csv(dsp, sep=',', encoding='utf-8') for dsp in dspaths]
  [i.shape for i in individual_dfs]
  return individual_dfs

dspaths = read_paths(dir_path)
individual_dfs = read_files(dspaths)

./MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
./MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv
./MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
./MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
./MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
./MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv
./MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
./MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv


In [4]:
individual_dfs[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191033 entries, 0 to 191032
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             191033 non-null  int64  
 1    Flow Duration                191033 non-null  int64  
 2    Total Fwd Packets            191033 non-null  int64  
 3    Total Backward Packets       191033 non-null  int64  
 4   Total Length of Fwd Packets   191033 non-null  int64  
 5    Total Length of Bwd Packets  191033 non-null  int64  
 6    Fwd Packet Length Max        191033 non-null  int64  
 7    Fwd Packet Length Min        191033 non-null  int64  
 8    Fwd Packet Length Mean       191033 non-null  float64
 9    Fwd Packet Length Std        191033 non-null  float64
 10  Bwd Packet Length Max         191033 non-null  int64  
 11   Bwd Packet Length Min        191033 non-null  int64  
 12   Bwd Packet Length Mean       191033 non-nul

In [ ]:
individual_dfs[1].sample()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,...,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Idle Mean,Idle Std,Label
18343,66,2,1,0,0,0.0,0.0,0.0,0.0,0.0,...,0,349,65535,0,32,0.0,0.0,0.0,0.0,BENIGN


In [ ]:
print(individual_dfs[1].isnull().sum())
print(individual_dfs[1].notna().sum())

NameError: name 'individual_dfs' is not defined

# Downsizing

In [5]:
individual_dfs[0].dtypes

 Destination Port                int64
 Flow Duration                   int64
 Total Fwd Packets               int64
 Total Backward Packets          int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
 Idle Std                      float64
 Idle Max                        int64
 Idle Min                        int64
 Label                          object
Length: 79, dtype: object

In [6]:
individual_dfs = [df_shrink(df) for df in individual_dfs]

In [ ]:
individual_dfs[0].dtypes

 Destination Port                 int32
 Flow Duration                    int32
 Total Fwd Packets                int32
 Total Backward Packets           int32
Total Length of Fwd Packets       int32
                                 ...   
Idle Mean                       float32
 Idle Std                       float32
 Idle Max                         int32
 Idle Min                         int32
 Label                         category
Length: 79, dtype: object

# Usuwanie niezdefiniowanych wartości

In [7]:
def drop_nan(individual_dfs):
  for df in individual_dfs:
      df.replace([np.inf, -np.inf], np.nan, inplace=True)
      # print(df.isna().any(axis=1).sum(), "rows with at least one NaN to remove")
      df.dropna(inplace=True)
  [i.shape for i in individual_dfs]
  return individual_dfs

individual_dfs = drop_nan(individual_dfs)

# Usuwanie zduplikowanych wierszy

In [8]:
def drop_dupes(individual_dfs):
  for df in individual_dfs:
      print(df.duplicated().sum(), "fully duplicate rows to remove")
      df.drop_duplicates(inplace=True)
      df.reset_index(inplace=True, drop=True)
  [i.shape for i in individual_dfs]
  return individual_dfs

individual_dfs = drop_dupes(individual_dfs)

26831 fully duplicate rows to remove
6867 fully duplicate rows to remove
35605 fully duplicate rows to remove
2629 fully duplicate rows to remove
72319 fully duplicate rows to remove
80914 fully duplicate rows to remove
6052 fully duplicate rows to remove
24019 fully duplicate rows to remove


## Zapisywanie danych w CSV

In [9]:
def convert(individual_dfs):
  for i, df in enumerate(individual_dfs):
        csv_path = f"./MachineLearningCVE{dspaths[i].split('/')[-1].replace('.pcap_ISCX', '')}"
        df.to_csv(csv_path, index=False)

convert(individual_dfs)

## Zapisywanie połączonych danych w CSV

In [ ]:
combined_df = pd.concat(individual_dfs, ignore_index=True)
combined_df.to_csv("combined.csv", index=False)


# Zapisywanie danych w parquet



In [ ]:
def convert(individual_dfs):
  for i, df in enumerate(individual_dfs):
      df.to_parquet(f"./MachineLearningCVE{dspaths[i].split('/')[-1].replace('.pcap_ISCX.csv', '.parquet')}")

convert(individual_dfs)

## Zapisywanie połączonych danych w parquet

In [ ]:
def convert_to_single_parquet(individual_dfs, output_path):
    combined_df = pd.concat(individual_dfs, ignore_index=True)
    combined_df.to_parquet(output_path)

convert_to_single_parquet(individual_dfs, "combined.parquet")

# Czytanie plików i misc

## Zamienianie wartośći z kolumny Label na BENINGN/ATTACK


In [ ]:
label_map = {'BENIGN': 'BENIGN', 'DDoS': 'ATTACK', 'DoS Hulk': 'ATTACK', 'DoS GoldenEye': 'ATTACK', 'DoS slowloris': 'ATTACK',
             'FTP-Patator': 'ATTACK', 'SSH-Patator':  'ATTACK', 'DoS Slowhttptest': 'ATTACK', 'PortScan': 'ATTACK',
             'Web Attack � Brute Force' : 'ATTACK', 'Bot': 'ATTACK', 'Web Attack � XSS': 'ATTACK', 'Infiltration': 'ATTACK',
             'Web Attack � Sql Injection': 'ATTACK', 'Heartbleed': 'ATTACK'}
combined01['Label'] = combined01['Label'].map(label_map).fillna('ATTACK')

print(combined01['Label'].value_counts())

Label
BENIGN    1886548
ATTACK     316445
Name: count, dtype: int64
